## Simulation Ramp (1.4 -> 1.6)

In [1]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

p_0 = 1.4;
p_1 = 1.6;
t_max  = 5.0;
p_ramp = 7.5e4;

#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.0_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.cell();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [2]:
#----Integrate using NPT

press_1 = hoomd.variant.linear_interp(points = [(0.0,p_0), (p_ramp, p_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 10.0, P = press_1);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 
hoomd.run(p_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 756088 / 831000 | TPS 8.67021 | ETA 02:24:00
Time 00:00:20 | Step 756167 / 831000 | TPS 7.68543 | ETA 02:42:17
Time 00:00:30 | Step 756253 / 831000 | TPS 8.4481 | ETA 02:27:27
Time 00:00:40 | Step 756345 / 831000 | TPS 9.1816 | ETA 02:15:30
Time 00:00:50 | Step 756437 / 831000 | TPS 8.95341 | ETA 02:18:47
Time 00:01:01 | Step 756531 / 831000 | TPS 9.33626 | ETA 02:12:56
Time 00:01:11 | Step 756628 / 831000 | TPS 9.6834 | ETA 02:08:00
Time 00:01:21 | Step 756734 / 831000 | TPS 10.4524 | ETA 01:58:25
Time 00:01:31 | Step 756840 / 831000 | TPS 10.5727 | ETA 01:56:54
Time 00:01:41 | Step 756937 / 831000 | TPS 9.65453 | ETA 02:07:51
Time 00:01:51 | Step 757028 / 831000 | TPS 9.087 | ETA 02:15:40
Time 00:02:01 | Step 757130 / 831000

**ERROR**: Particle with unique tag 399 is no longer in the simulation box.

**ERROR**: Cartesian coordinates: 
**ERROR**: x: 52.1176 y: -78.0516 z: 18.9047
**ERROR**: Fractional coordinates: 
**ERROR**: f.x: 2.82016 f.y: -2.97468 f.z: 1.34159
**ERROR**: Local box lo: (-11.2315, -11.2315, -11.2315)
**ERROR**:           hi: (11.2315, 11.2315, 11.2315)


RuntimeError: Error computing cell list

In [ ]:
system.get_metadata()